In [96]:
# load dataset from kaggle url
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# !ls

In [97]:
df=pd.read_csv("Amazon-Products.csv")

In [98]:
df.columns

Index(['Unnamed: 0', 'name', 'main_category', 'sub_category', 'image', 'link',
       'ratings', 'no_of_ratings', 'discount_price', 'actual_price'],
      dtype='object')

In [99]:
df.describe()

,Unnamed: 0
count,516684.000000
mean,6805.774835
std,5689.098661
min,0.000000
25%,1449.750000
50%,5637.000000
75%,11066.000000
max,19199.000000


Unnamed is id of product, renaming it

In [100]:
df.rename(columns={"Unnamed: 0":"product_id"},inplace=True)

In [101]:
df.sample(3)

,product_id,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price
484179,5049,PRO GYM Men's Solid Regular Dry Fit Gym T-Shir...,men's clothing,T-shirts & Polos,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/PRO-GYM-Regular-T-Shirt-...,5.0,1,₹245,₹999
30637,493,"Qualtos Women's Slippers (Yellow Laugh, 7.5-8....",kids' fashion,Baby Fashion,https://m.media-amazon.com/images/I/71iBUSoj-i...,https://www.amazon.in/Qualtos-Yellow-Bedroom-N...,3.9,37,₹399,₹999
12047,479,"Mivi Roam 2 Bluetooth 5W Portable Speaker,24 H...","tv, audio & cameras",All Electronics,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Mivi-Wireless-Bluetooth-...,4.3,"36,230","₹1,199","₹2,999"


In [102]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 516684 entries, 0 to 516683
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   product_id      516684 non-null  int64 
 1   name            516684 non-null  object
 2   main_category   516684 non-null  object
 3   sub_category    516684 non-null  object
 4   image           516683 non-null  object
 5   link            516683 non-null  object
 6   ratings         352850 non-null  object
 7   no_of_ratings   352850 non-null  object
 8   discount_price  458180 non-null  object
 9   actual_price    499395 non-null  object
dtypes: int64(1), object(9)
memory usage: 39.4+ MB


### ratings , no_of_ratings, discount_price and actual_price has missing values.   

ratings , no_of_ratings, discount_price and actual_price are integer or decimal value columns but are stored as object type in pandas dataframe.
We need to convert them to proper data types.

In [103]:
df['discount_price'].unique()

array(['₹32,999', '₹46,490', '₹34,490', ..., '₹12,596.50', '₹7,535',
       '₹39,734'], dtype=object)

',' and ₹ need to be removed to make it decimal data type .

In [104]:
df['no_of_ratings'].dtypes

dtype('O')

In [105]:
df['no_of_ratings'].unique()

array(['2,255', '2,948', '1,206', ..., '4,528',
       'Usually dispatched in 1 to 2 months.', '10,685'], dtype=object)

no_of_ratings column has ',' which is causing it to become object type

Convert no_of _ratings into  float type

In [107]:
df['no_of_ratings']=df['no_of_ratings'].apply(lambda x: x.replace(',','') if "," in str(x) else x)

In [108]:
# df['no_of_ratings']=df['no_of_ratings'].astype(float)
# some error ocurred

ValueError: could not convert string to float: 'Only 2 left in stock.'

In [110]:
df.groupby(["no_of_ratings"])["no_of_ratings"].value_counts().tail(25)

no_of_ratings
998                                       18
9984                                       3
9985                                       1
9986                                       3
999                                        9
9995                                       1
FREE Delivery by Amazon                 2679
Only 1 left in stock.                    847
Only 2 left in stock.                    524
Only 3 left in stock.                     15
Only 4 left in stock.                     15
Only 5 left in stock.                      6
Usually dispatched in 1 to 2 months.       1
Usually dispatched in 11 to 12 days.      17
Usually dispatched in 2 to 3 days.         6
Usually dispatched in 2 to 3 weeks.        8
Usually dispatched in 3 to 4 weeks.      847
Usually dispatched in 3 to 5 days.         3
Usually dispatched in 4 to 5 days.       190
Usually dispatched in 4 to 5 weeks.        8
Usually dispatched in 5 to 6 days.        70
Usually dispatched in 6 to 7 days.       

There are some records which have wrong data in no_of_ratings column, so imputing them with nan values

In [120]:
wrong_strings_list=['FREE Delivery by Amazon','Only 1 left in stock.','Only 2 left in stock.',
                    'Only 3 left in stock.','Usually dispatched in 2 to 3 days.','Usually dispatched in 3 to 4 weeks.',
                    'Usually dispatched in 4 to 5 days.','Usually dispatched in 4 to 5 weeks.','Usually dispatched in 5 to 6 days.'
                    ,'Usually dispatched in 6 to 7 days.','Usually dispatched in 7 to 8 days.','Only 4 left in stock.'
                    ,'Only 5 left in stock.','Usually dispatched in 11 to 12 days.','Usually dispatched in 3 to 5 days.'
                    ,'Usually dispatched in 9 to 10 days.','Usually dispatched in 2 to 3 weeks.','Usually dispatched in 8 to 9 days.'
                    ,'Usually dispatched in 1 to 2 months.']

In [121]:
# replacing each wrong string to null values
for i in wrong_strings_list:
  df.loc[df['no_of_ratings']==i, 'no_of_ratings'] = np.nan


In [122]:
df['no_of_ratings']=df['no_of_ratings'].astype(float)

For ratings column

In [124]:
df.groupby(["ratings"])["ratings"].value_counts().tail(15)

ratings
4.4       13312
4.5       12031
4.6        7941
4.7        5116
4.8        2234
4.9         490
5.0       30564
FREE       1350
Get        3951
₹100          1
₹2.99         2
₹65           1
₹68.99        1
₹70           5
₹99          14
Name: count, dtype: int64

In [126]:
wrong_strings_list=['FREE','Get','₹100'
                    ,'₹2.99','₹65','₹68.99'
                    ,'₹70','₹99'
                    ]

# replacing each wrong string to null values
for i in wrong_strings_list:
  df.loc[df['ratings']==i, 'ratings'] = np.nan


In [127]:
df['ratings']=df['ratings'].astype(float)

For discount price and actual price

In [132]:
df["discount_price"].value_counts()

discount_price
₹499       16930
₹299       13682
₹399       12888
₹999       12591
₹599       10851
           ...  
₹26,351        1
₹19,220        1
₹13,043        1
₹42,654        1
₹39,734        1
Name: count, Length: 26831, dtype: int64

In [138]:
df["discount_price"]=df["discount_price"].apply(lambda x: x.replace("₹","") if "₹" in str(x)  else x)
df["discount_price"]=df["discount_price"].apply(lambda x: x.replace(",","") if "," in str(x)  else x)
df["discount_price"]=df["discount_price"].astype(float)

In [139]:
df['discount_price'].value_counts()

discount_price
499.0      16930
299.0      13682
399.0      12888
999.0      12591
599.0      10851
           ...  
26351.0        1
19220.0        1
13043.0        1
42654.0        1
39734.0        1
Name: count, Length: 26831, dtype: int64

Repeating the same preprocessing For actual price column

In [140]:
df["actual_price"].value_counts()

actual_price
₹999       44985
₹1,999     23568
₹1,499     16976
₹499       13648
₹2,999     13200
           ...  
₹50,188        1
₹44,294        1
₹42,056        1
₹60,592        1
₹52,934        1
Name: count, Length: 22973, dtype: int64

In [141]:
df["actual_price"]=df["actual_price"].apply(lambda x: x.replace("₹","") if "₹" in str(x)  else x)
df["actual_price"]=df["actual_price"].apply(lambda x: x.replace(",","") if "," in str(x)  else x)
df["actual_price"]=df["actual_price"].astype(float)

In [142]:
df.describe()

,product_id,ratings,no_of_ratings,discount_price,actual_price
count,516684.000000,347525.000000,347525.000000,4.581800e+05,4.993950e+05
mean,6805.774835,3.835530,884.167951,2.732520e+03,2.453841e+04
std,5689.098661,0.755252,8917.385957,9.749005e+03,1.400946e+07
min,0.000000,1.000000,1.000000,8.000000e+00,0.000000e+00
25%,1449.750000,3.500000,4.000000,3.990000e+02,9.900000e+02
50%,5637.000000,3.900000,21.000000,6.990000e+02,1.599000e+03
75%,11066.000000,4.300000,139.000000,1.487000e+03,3.199000e+03
max,19199.000000,5.000000,589547.000000,1.249990e+06,9.900000e+09


In [143]:
# df.query('(ratings == "FREE") | (ratings == "GET") |  (ratings == "₹68.99") ' ).sample(5)

In [19]:
df.sample(4)

,product_id,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price
22604,260,"Saffola Honey Active, Made with Sundarban Fore...",grocery & gourmet foods,All Grocery & Gourmet Foods,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Saffola-Honey-100-Pure-5...,3.8,4081,₹194,₹220
39158,2486,"RIDE OR DIE, GEAR - keychain for bikes - 2 NEW...",accessories,Bags & Luggage,https://m.media-amazon.com/images/I/71D3oIZLWk...,https://www.amazon.in/RIDE-DIE-GEAR-SBKOFFICIA...,3.8,34,₹199,₹299
42184,5512,Gear polyester 26 Cms Travel Duffle(DUFJCQURD1...,accessories,Bags & Luggage,https://m.media-amazon.com/images/I/71H2ORQj2B...,https://www.amazon.in/Gear-Jacquard-Fashion-tr...,4.1,46,₹719,"₹1,499"
2764,2044,"Casa Copenhagen Lion Collection, 40 L - Person...",appliances,All Appliances,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/Casa-Copenhagen-Lion-Col...,4.5,50,"₹5,999","₹19,999"


In [20]:
df.main_category.value_counts()

main_category
accessories                12553
appliances                 10296
tv, audio & cameras         9600
sports & fitness            3600
toys & baby products        2448
stores                      2352
kids' fashion               2208
bags & luggage              2064
car & motorbike             1272
home & kitchen              1224
grocery & gourmet foods      960
pet supplies                 648
Name: count, dtype: int64

In [21]:
percentage=(df.main_category.value_counts().head(5).sum())/len(df)*100
percentage

78.20619603859828

Only top 5 main categories cover 77% of all products

In [22]:
df.groupby(["main_category"])["sub_category"].value_counts()

main_category            sub_category                  
accessories              Bags & Luggage                    12553
appliances               All Appliances                     9576
                         Air Conditioners                    720
bags & luggage           Backpacks                          2064
car & motorbike          All Car & Motorbike Products       1272
grocery & gourmet foods  All Grocery & Gourmet Foods         960
home & kitchen           All Home & Kitchen                 1224
kids' fashion            Baby Fashion                       2208
pet supplies             All Pet Supplies                    648
sports & fitness         All Sports, Fitness & Outdoors     1224
                         Badminton                          1200
                         All Exercise & Fitness             1176
stores                   Amazon Fashion                     2352
toys & baby products     Baby Bath, Skin & Grooming         1392
                         Baby Prod

Each main category has one to two sub categories. Interestingly sports & fitness is a special main category that has 3 sub categories.

In [23]:
df.duplicated().value_counts()

False    49225
Name: count, dtype: int64

No duplicate present in our dataset